In [70]:
import numpy as np
import pandas as pd

In [71]:
# Define the true parameters for the datasets
m = np.vstack((np.zeros(5), np.ones(5)))  # Mean vectors for class 1 and class 2
# print(m, m.shape)
s = np.zeros((2,5,5))  # Covariance matrices for class 1 and class 2
s[0,:,:] = np.array([[0.8, 0.2, 0.1, 0.05, 0.01],   # Covariance matrix for class 1
                    [0.2, 0.7, 0.1, 0.03, 0.02],
                    [0.1, 0.1, 0.8, 0.02, 0.01],
                    [0.05, 0.03, 0.02, 0.9, 0.01],
                    [0.01, 0.02, 0.01, 0.01, 0.8]])
s[1,:,:] = np.array([[0.9, 0.1, 0.05, 0.02, 0.01],   # Covariance matrix for class 2
                    [0.1, 0.8, 0.1, 0.02, 0.02],
                    [0.05, 0.1, 0.7, 0.02, 0.01],
                    [0.02, 0.02, 0.02, 0.6, 0.02],
                    [0.01, 0.02, 0.01, 0.02, 0.7]])    
# print(s, s.shape)
p = np.vstack((1/2, 1/2))  # Class probabilities
# print(p, p.shape)

In [73]:
def NaiveBayesCLF(trainfile, testfile, x, m, s, p):
    # open the datasets
    df = pd.read_excel(trainfile, header=None ,names=('x1', 'x2', 'x3', 'x4', 'x5', 'label'))
    df_test = pd.read_excel(testfile, header=None)
    df_testy = df_test.iloc[:, -1].to_numpy()
    df_testx = df_test.iloc[:, :-1].to_numpy()
    
    # estimate the prior
    priors = (df.groupby('label').size() / len(df)).to_numpy()
    # print(priors)
    # estimate the means
    means = df.groupby('label').mean().to_numpy()
    # estimate the variances
    vars = df.groupby('label').var().to_numpy()
    # print(vars)
    # store possible classes
    classes = df['label'].unique()
    # print(classes)
    
    # get the test scores
    tst_z = gaussian(df_testx, means, vars)
    # multiply by priors
    probs = tst_z * priors
    # class predictions
    pred_y = 1 + np.argmax(probs, axis=1)
    
    # compute the error
    error = np.mean(pred_y != df_testy)
    return error

In [66]:
def gaussian(X, mu, var):
    # Calculate the exponent term
    exponent = -((X[:, np.newaxis] - mu) ** 2) / (2 * var)
    
    # Calculate the Gaussian probability density function
    pdf = np.exp(exponent) / np.sqrt(2 * np.pi * var)
    
    # Compute the product of probabilities along each feature axis
    return np.prod(pdf, axis=2)


In [67]:
# Create an empty DataFrame
errors = pd.DataFrame(index=['Naive Bayes', 'MLE Bayes', 'Bayes'], 
                      columns=['Error Train_N = 100', 'Error Train_N = 1000'])

# Optionally, you can fill the DataFrame with NaN values
errors = errors.fillna(float('nan'))
errors

,Error Train_N = 100,Error Train_N = 1000
Naive Bayes,NaN,NaN
MLE Bayes,NaN,NaN
Bayes,NaN,NaN


In [68]:
errors.at['Naive Bayes', 'Error Train_N = 100'] = NaiveBayesCLF('Proj4Train100.xlsx', 'Proj4Test.xlsx')
errors.at['Naive Bayes', 'Error Train_N = 1000'] = NaiveBayesCLF('Proj4Train1000.xlsx', 'Proj4Test.xlsx')

In [69]:
errors

,Error Train_N = 100,Error Train_N = 1000
Naive Bayes,0.1248,0.1213
MLE Bayes,NaN,NaN
Bayes,NaN,NaN
